In [3]:
!pip install pyspark

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, to_date, month, year, sum, avg, count
from pyspark.sql.window import Window
from pyspark.sql.functions import datediff, current_date

# Initialize Spark session
spark = SparkSession.builder.appName("Retail Sales Data Analysis").getOrCreate()


In [4]:
sales_df = spark.read.csv("/content/sample_data/Furniture_Sales_Data.csv", header=True, inferSchema=True)


In [5]:
sales_df.createOrReplaceTempView("sales_data")
sales_df.printSchema()
sales_df.show(10)

root
 |-- OrderID: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- ProductNames: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- Region: string (nullable = true)

+---------+----------+------------+--------+-------+----------+---------+
|  OrderID|CustomerID|ProductNames|Quantity|  Price| OrderDate|   Region|
+---------+----------+------------+--------+-------+----------+---------+
|ORD100000|  CUST9055|   Bookshelf|       2|1865.65|2023-03-12|   France|
|ORD100001|  CUST1538|        Sofa|       4|1726.87|2024-03-08|    China|
|ORD100002|  CUST6940|Coffee Table|       4| 760.89|2022-06-20|    China|
|ORD100003|  CUST1625|     Dresser|       5| 368.03|2023-05-01|   France|
|ORD100004|  CUST9204|    TV Stand|       1| 753.76|2023-07-03|    India|
|ORD100005|  CUST6897|    Recliner|       2|1042.94|2024-07-17|Australia|
|ORD100006|  CUST3475|Coffee Table|       6| 753

Dataset schema with field names, data types, and nullability. Key attributes: OrderID, CustomerID, ProductNames, Quantity, Price, OrderDate, and Region.

First 10 rows displaying order details, used for initial inspection and data quality checks.

In [6]:
from pyspark.sql.functions import col

# Contar valores nulos por columna
sales_df.select([col(c).isNull().cast("int").alias(c) for c in sales_df.columns]).summary("count").show()


+-------+-------+----------+------------+--------+-----+---------+------+
|summary|OrderID|CustomerID|ProductNames|Quantity|Price|OrderDate|Region|
+-------+-------+----------+------------+--------+-----+---------+------+
|  count|   1500|      1500|        1500|    1500| 1500|     1500|  1500|
+-------+-------+----------+------------+--------+-----+---------+------+



Total count of records for each column: 1,500 entries with no missing values. Ensures data completeness for accurate analysis.

In [7]:
sales_df = sales_df.dropna()


In [8]:
sales_df.printSchema()


root
 |-- OrderID: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- ProductNames: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- Region: string (nullable = true)



In [9]:
# Data Cleaning
sales_df = sales_df.dropna() # Drop rows with missing values
sales_df = sales_df.withColumn("Quantity", col("Quantity").cast("integer"))
sales_df = sales_df.withColumn("Price", col("Price").cast("double"))
sales_df = sales_df.withColumn("OrderDate", to_date(col("OrderDate"), "yyyy-MM-dd"))
sales_df = sales_df.dropDuplicates()


In [13]:
sales_df.describe(["Quantity", "Price"]).show()


+-------+-----------------+------------------+
|summary|         Quantity|             Price|
+-------+-----------------+------------------+
|  count|             1500|              1500|
|   mean|5.460666666666667|1037.9930733333313|
| stddev|2.877146422513098| 568.0992794641094|
|    min|                1|             51.93|
|    max|               10|           1999.52|
+-------+-----------------+------------------+



Mean Quantity: 5.46

Mean price: $1,037.99

Min-Max Quantity: 1-10

Min-Max price: $51.93- $1,999.52

Essential for understanding sales distribution and price variations.

In [11]:
# 1. Total sales per product
total_sales_per_product = sales_df.groupBy("ProductNames").agg(sum("Price").alias("TotalSales")) # Changed "Product" to "ProductNames"
total_sales_per_product.orderBy(col("TotalSales").desc()).show()

+------------+------------------+
|ProductNames|        TotalSales|
+------------+------------------+
|    Recliner|176088.29999999984|
|Office Chair|175345.29999999996|
|    Wardrobe|163726.85000000003|
|Coffee Table|157625.70000000004|
|   Bed Frame|152786.90999999997|
|Dining Table| 150707.9300000001|
|   Bookshelf|          149913.7|
|        Sofa|147653.82999999996|
|    TV Stand|143703.18000000005|
|     Dresser|         139437.91|
+------------+------------------+



Highest revenue-generating products:

Recliner: $176,088.30

Office Chair: $175,345.30

Wardrobe: $163,726.85

Identifies key revenue drivers for business decisions.

In [15]:
# 2. Products with the most units sold
top_selling_products = sales_df.groupBy("ProductNames").agg(sum("Quantity").alias("TotalUnitsSold"))
top_selling_products.orderBy(col("TotalUnitsSold").desc()).show()

+------------+--------------+
|ProductNames|TotalUnitsSold|
+------------+--------------+
|Office Chair|           921|
|   Bookshelf|           882|
|    Wardrobe|           882|
|    Recliner|           872|
|Coffee Table|           856|
|   Bed Frame|           835|
|        Sofa|           783|
|Dining Table|           744|
|    TV Stand|           709|
|     Dresser|           707|
+------------+--------------+



Office Chair – 921 units

Bookshelf / Wardrobe – 882 units

Recliner – 872 units

Crucial for demand forecasting and inventory planning.


In [16]:
# 3. Revenue per region
revenue_per_region = sales_df.groupBy("Region").agg(sum("Price").alias("TotalRevenue"))
revenue_per_region.orderBy(col("TotalRevenue").desc()).show()

+---------+------------------+
|   Region|      TotalRevenue|
+---------+------------------+
|    Japan|176477.73999999996|
|   Brazil|         163008.67|
|Australia|161849.93000000005|
|    China|159317.91999999995|
|      USA|156733.82000000007|
|   France| 155944.9800000001|
|   Canada|152472.43000000005|
|  Germany|145792.34999999995|
|    India|142958.74999999997|
|       UK|         142433.02|
+---------+------------------+



Top revenue-generating regions:

Japan – $176,477

Brazil – $163,008

Australia – $161,849

Helps in identifying strong markets and optimizing sales strategies.

In [17]:
# 4. Monthly and yearly sales patterns
sales_df = sales_df.withColumn("Month", month("OrderDate"))
sales_df = sales_df.withColumn("Year", year("OrderDate"))

In [44]:
# 5. Customer segmentation (RFM - Recency, Frequency, Monetary Value)
sales_df = sales_df.withColumn("TotalValue", col("Quantity")*col("Price"))
customer_rfm = sales_df.groupBy("CustomerID").agg(
    F.max("OrderDate").alias("Recency"),
    F.count("OrderDate").alias("Frequency"),
    F.sum("TotalValue").alias("Monetary")
)
customer_rfm.show()


+----------+----------+---------+------------------+
|CustomerID|   Recency|Frequency|          Monetary|
+----------+----------+---------+------------------+
|  CUST1580|2022-08-02|        1|           1446.38|
|  CUST6280|2021-11-26|        1|          17541.72|
|  CUST2052|2024-04-11|        1|           5572.42|
|  CUST5060|2022-04-06|        1|            6226.6|
|  CUST6524|2021-12-05|        1|           10659.7|
|  CUST1841|2023-06-10|        1|          10473.54|
|  CUST9366|2021-11-05|        1|           5474.28|
|  CUST3892|2024-06-19|        2|11104.990000000002|
|  CUST4893|2022-01-12|        1|             685.2|
|  CUST9014|2022-08-28|        1|1905.4499999999998|
|  CUST6602|2021-12-29|        1|370.78999999999996|
|  CUST3343|2024-04-20|        1|            756.38|
|  CUST3004|2022-10-18|        1|            3086.8|
|  CUST9662|2024-03-22|        1|           1854.34|
|  CUST8087|2022-12-10|        1|16986.690000000002|
|  CUST3789|2023-01-10|        2|14530.0799999

Recency: Last purchase date

Frequency: Transactions per customer

Monetary: Total spending

Useful for customer retention strategies and targeted marketing.

In [30]:
# 6. Average order value
avg_order_value = sales_df.groupBy("OrderID").agg(sum("Price").alias("OrderValue"))
avg_order_value.agg(avg("OrderValue")).show()

+------------------+
|   avg(OrderValue)|
+------------------+
|1037.9930733333342|
+------------------+



$1,037.99 per order. Helps in pricing optimization and revenue forecasting.

In [32]:
# 7. Product performance by region
product_region_sales = sales_df.groupBy("ProductNames", "Region").agg(sum("Price").alias("TotalSales"))
product_region_sales.orderBy("ProductNames", "Region").show()

+------------+---------+------------------+
|ProductNames|   Region|        TotalSales|
+------------+---------+------------------+
|   Bed Frame|Australia|          16723.84|
|   Bed Frame|   Brazil| 9483.230000000001|
|   Bed Frame|   Canada|           20650.0|
|   Bed Frame|    China|           25551.3|
|   Bed Frame|   France|           9950.59|
|   Bed Frame|  Germany|          13291.15|
|   Bed Frame|    India|14864.009999999998|
|   Bed Frame|    Japan|15411.599999999999|
|   Bed Frame|       UK|          12690.32|
|   Bed Frame|      USA|          14170.87|
|   Bookshelf|Australia|12428.220000000001|
|   Bookshelf|   Brazil|19935.249999999996|
|   Bookshelf|   Canada|14800.149999999998|
|   Bookshelf|    China|           9623.79|
|   Bookshelf|   France|23912.000000000004|
|   Bookshelf|  Germany|14340.610000000002|
|   Bookshelf|    India|           4999.28|
|   Bookshelf|    Japan|21185.899999999998|
|   Bookshelf|       UK|          12969.84|
|   Bookshelf|      USA|15718.65

Breakdown of total sales per product by region. Supports regional inventory management and market-specific strategies.

In [41]:
# 8. Identify low-performing products
low_performing = sales_df.groupBy("ProductNames").agg(sum("Quantity").alias("TotalUnitsSold")).filter("TotalUnitsSold < 500")
low_performing.show()

+------------+--------------+
|ProductNames|TotalUnitsSold|
+------------+--------------+
+------------+--------------+



### **Low-Performing Products**  
Identifies products with total unit sales below **500** using a threshold-based filtering approach.
The query groups sales by `ProductNames`, calculates total units sold, and applies a filter condition (`TotalUnitsSold < 500`).
The empty result indicates that all products sold at least **500 units**, meaning no items fall into the low-performance category.

In [36]:
# 9. Days with highest sales
daily_sales = sales_df.groupBy("OrderDate").agg(sum("Price").alias("DailySales"))
daily_sales.orderBy(col("DailySales").desc()).show()


+----------+------------------+
| OrderDate|        DailySales|
+----------+------------------+
|2024-03-20| 7368.860000000001|
|2024-06-02|           7199.34|
|2023-11-24|           7183.07|
|2021-10-28|           7138.58|
|2021-10-27| 7130.210000000001|
|2024-01-05|           6228.87|
|2021-11-01|           6221.28|
|2022-11-23|           5909.39|
|2024-08-22|           5738.38|
|2022-03-07|           5611.24|
|2022-07-03|           5558.91|
|2023-05-05|           5479.41|
|2022-08-28|           5436.46|
|2021-10-18| 5392.509999999999|
|2022-03-23|           5248.09|
|2024-05-07|           5066.21|
|2024-03-14| 5034.299999999999|
|2022-04-04| 5031.549999999999|
|2022-03-22|           4938.22|
|2022-09-04|4904.6900000000005|
+----------+------------------+
only showing top 20 rows



### **Days with Highest Sales**  
Aggregates total revenue per day by summing `Price` values for each `OrderDate`, then sorts results in descending order to identify peak sales days.
The highest recorded daily sales occurred on March 20, 2024, with $7,368.86 in revenue.

The top 20 days all exceed **$4,900**, providing insight into peak demand periods.

In [43]:
# 10. Stock level suggestions (simplified example)
# Calculate a moving average for the past 30 days (adjust window as needed)
w = Window.partitionBy("ProductNames").orderBy("OrderDate").rowsBetween(-30, 0)  # Consider previous 30 days

sales_with_moving_average = sales_df.withColumn(
    "MovingAverage", avg("Price").over(w)
)

sales_with_moving_average.show()


+---------+----------+------------+--------+-------+----------+---------+-----+----+-----------------+------------------+
|  OrderID|CustomerID|ProductNames|Quantity|  Price| OrderDate|   Region|Month|Year|       TotalValue|     MovingAverage|
+---------+----------+------------+--------+-------+----------+---------+-----+----+-----------------+------------------+
|ORD100199|  CUST9624|   Bed Frame|       1|1605.58|2021-10-18|       UK|   10|2021|          1605.58|           1605.58|
|ORD100646|  CUST8108|   Bed Frame|       9|1047.21|2021-10-24|Australia|   10|2021|          9424.89|          1326.395|
|ORD100685|  CUST3387|   Bed Frame|       2| 1660.6|2021-10-27|       UK|   10|2021|           3321.2|1437.7966666666664|
|ORD100948|  CUST7545|   Bed Frame|       4|1516.58|2021-10-31|      USA|   10|2021|          6066.32|1457.4924999999998|
|ORD100187|  CUST2068|   Bed Frame|       4|1519.16|2021-11-03|    India|   11|2021|          6076.64|1469.8259999999998|
|ORD100242|  CUST2981|  

Analyzing TotalValue and MovingAverage for Bed Frame sales across different regions and time periods provides insights for inventory planning.

High Demand Periods: Peaks in TotalValue suggest increased sales activity. For instance, large transactions in October-December 2021 indicate seasonal demand.
Moving Average Trends: Helps smooth fluctuations and predict upcoming demand. Lower values in early 2022 suggest a decline, signaling a potential need for stock adjustments.
Recommendation: Maintain higher stock levels during Q4 due to historical peak demand. Monitor early-year trends to avoid overstocking and optimize warehouse space.

# Sección nueva